
"""
Created on Mon Sep 19 16:57:47 2016 | modified & converted to notebook

@author: hrm

Exploring MTA Turnstile Data

%% Refs
http://web.mta.info/developers/turnstile.html
https://data.ny.gov/Transportation/NYC-Transit-Subway-Entrance-And-Exit-Data/i9wp-a4ja/data
https://spatialityblog.com/2010/07/08/mta-gis-data-update/


%% Pandas
http://pythonhow.com/accessing-dataframe-columns-rows-and-cells

"""


In [1]:
import matplotlib
%matplotlib inline

In [ ]:
## Make into Function -- load list of url data
# urlList = ["http://web.mta.info/developers/data/nyct/turnstile/turnstile_160604.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160528.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160521.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160514.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160507.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160430.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160423.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160409.txt",
#            "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt"]

#len(urlList)=9

# import pandas as pd
# #http://pandas.pydata.org/pandas-docs/stable/merging.html

# cnt=0
# for u in urlList:
#     Dtmp = pd.read_csv(u)
#     if cnt==0:
#         Dmat = Dtmp
#     else:
#         Dmat = pd.concat([Dmat, Dtmp])
#     cnt += 1

# del Dtmp

# D= Dmat

In [2]:
import pandas as pd

#url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt"
url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160409.txt"
D = pd.read_csv(url,na_values=['.', 0])
#print(D)
#D.head

#D.columns
#Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME','DESC', 'ENTRIES','EXITS'], dtype='object')

In [3]:
import dateutil.parser as DUparser
from collections import defaultdict

Ddict = defaultdict(list)
for row in D.values:
    ca = row[0]
    unit = row[1]
    scp = row[2]
    station = row[3]
    linename = row[4]
    division = row[5]
    date = row[6]
    #date_wday = DUparser.parse(date).weekday()
    time = row[7]
    desc = row[8]
    entries = row[9]
    exits = row[10]

#    Ddict[(ca, unit, scp, station)].append([DUparser.parse(date + " " + time), date_wday, int(entries) ]) #,  int(exits)])

    Ddict[(ca, unit, scp, station)].append([DUparser.parse(date + " " + time), int(entries) ]) #,  int(exits)])

#    Ddict2[(ca, unit, scp, station, linename)].append([DUparser.parse(date + " " + time), int(entries) ])



In [ ]:
#==============================================================================
# some time interval [0,4,8,12,16,20,(0)]+1 discrepancies over daylight savings but also sampling times.
# however, if we take day differences in the list of commuter vol/flow count data, this might not be an issue -- day+1[0] - day[0] || or Count_8/9pm - Count_4/8am? || TimeInt[5or-1] - TimeInt[1]

#https://pymotw.com/3/datetime/

#http://travel.mtanyct.info/serviceadvisory/

#==============================================================================

In [5]:
## Count Passenger Vol by Station-Unit_SCP
import numpy as np

Sdict = defaultdict(list)

for key in Ddict:
# new dict with station identifier as key and all entry/exit counts as list of values
    tmpHdict = defaultdict(list)

    for tf in Ddict[key]:
        DateTime= tf[0]
        #wday = tf[1]
        volume = tf[1]

        tmpHdict[DateTime.date()].append(volume)
#        break
#        print(tmpTdict)

        ## get difference between relevant highest(latest | 8/9pm) & lowest (earliest | 4/5am) index of count of the day
    tmpCdict = defaultdict(list)
    for date in tmpHdict.keys():
#        print (date)
        #count = max(tmpTdict[date]) - min(tmpTdict[date])
        if (len(tmpHdict[date])>=2):
            count = tmpHdict[date][len(tmpHdict[date])-1] - tmpHdict[date][1]
        else:
            count = [0]

        # tmpCdict[date].append([date.weekday(),count])
        tmpCdict[date.weekday()].append(count)
        
    if np.all(np.array(list(tmpCdict.values()))>0)==True:
        Sdict[key]= tmpCdict

#Sdict

In [ ]:
Sdict

In [ ]:
# list(Sdict.values())[0]

In [ ]:
# import numpy as np
# test=np.array(list(list(Sdict.values())[0].values()))
# np.all(test>0)

In [6]:
## Cluster Count Passenger Vol by Station

#DC = {(keys[0],keys[1],keys[3]): values for keys, values in Sdict.items() }


#pd.DataFrame(Sdict[list(Sdict.keys())[0]])
#pd.DataFrame(Sdict[list(Sdict.keys())[1]])
#pd.concat([])

SNdict0 = defaultdict(list)
SNdict = defaultdict(list)

for key in Sdict:
# new dict with stationName (+2other) identifier as key and all entry/exit counts as list of values
#    print key
#    print
    SNdict0[key[0],key[1],key[3]].append( pd.DataFrame(Sdict[key]) )
    SNdict[key[3]].append( pd.DataFrame(Sdict[key]) )


In [12]:
list(SNdict0.keys())

[('N503', 'R021', '42 ST-BRYANT PK'),
 ('N034', 'R334', 'CATHEDRAL PKWY'),
 ('N417', 'R269', 'BEDFORD-NOSTRAN'),
 ('R421', 'R427', 'MIDDLETOWN RD'),
 ('N317', 'R267', '46 ST'),
 ('N334B', 'R341', '75 AV'),
 ('R525', 'R018', '74 ST-BROADWAY'),
 ('N202', 'R315', '155 ST'),
 ('R174', 'R034', '125 ST'),
 ('N043', 'R186', '86 ST'),
 ('K026', 'R100', 'METROPOLITAN AV'),
 ('R401', 'R445', '3 AV 138 ST'),
 ('H012', 'R268', 'LORIMER ST'),
 ('N126', 'R441', 'VAN SICLEN AVE'),
 ('E014', 'R374', 'BAY PKWY'),
 ('N120A', 'R153', 'UTICA AV'),
 ('N039', 'R251', '96 ST'),
 ('N531', 'R129', 'BERGEN ST'),
 ('R317', 'R408', 'SIMPSON ST'),
 ('B031', 'R172', 'BRIGHTON BEACH'),
 ('R242', 'R049', '51 ST'),
 ('N037', 'R314', '103 ST'),
 ('N087', 'R282', 'SPRING ST'),
 ('J007', 'R377', 'FLUSHING AV'),
 ('R622', 'R123', 'FRANKLIN AV'),
 ('N049', 'R084', '59 ST COLUMBUS'),
 ('K024', 'R403', 'FOREST AVE'),
 ('N520', 'R240', 'GRAND ST'),
 ('N009', 'R174', '181 ST'),
 ('R416', 'R245', 'ST LAWRENCE AV'),
 ('D012', 'R

In [13]:
list(SNdict.keys())

['9 AV',
 'ELDER AV',
 'SUTPHIN-ARCHER',
 'INTERVALE AV',
 '42 ST-BRYANT PK',
 'WALL ST',
 'VAN SICLEN AV',
 'JACKSON AV',
 'OZONE PK LEFFRT',
 'HALSEY ST',
 '21 ST-QNSBRIDGE',
 '18 AV',
 '168 ST',
 'EUCLID AV',
 'JAY ST-METROTEC',
 'AVENUE P',
 'NECK RD',
 '181 ST',
 '34 ST-HUDSON YD',
 'UTICA AV',
 'COURT SQ-23 ST',
 '167 ST',
 'BROAD CHANNEL',
 'BUHRE AV',
 'HEWES ST',
 'PATH WTC',
 '40 ST LOWERY ST',
 '116 ST-COLUMBIA',
 'ELMHURST AV',
 '21 ST',
 'EASTCHSTER/DYRE',
 'KINGSTON-THROOP',
 '79 ST',
 'RALPH AV',
 'DEKALB AV',
 'ST. GEORGE',
 'BROOKLYN BRIDGE',
 'BRIARWOOD',
 'CENTRAL AV',
 '50 ST',
 'NORTHERN BLVD',
 '145 ST',
 'HUNTS POINT AV',
 '6 AV',
 '3 AV-149 ST',
 'MYRTLE AV',
 'BROADWAY',
 'TWENTY THIRD ST',
 "E 143/ST MARY'S",
 'BOWLING GREEN',
 'PROSPECT PARK',
 '57 ST',
 'CENTRAL PK N110',
 '169 ST',
 'JAMAICA VAN WK',
 'AVENUE X',
 '34 ST-HERALD SQ',
 'CARROLL ST',
 'CORTELYOU RD',
 'CLASSON AV',
 'LAFAYETTE AV',
 '174 ST',
 'PARKSIDE AV',
 '5 AV/53 ST',
 'FRANKLIN AV',
 'GA

In [8]:
SNdict

In [ ]:
SNdict[list(SNdict.keys())[0]]

In [11]:
pd.concat(SNdict[list(SNdict.keys())[0]])

# list(SNdict.keys())[100]
# list(SNdict.keys())[1]
# list(SNdict.keys())[0]


,0,1,2,3,4,5,6
0,1240,1386,1438,1371,1363,876,727
0,1461,1469,1495,1502,1535,964,818
0,1344,1322,1373,1322,1351,828,718
0,478,490,439,461,435,239,208
0,1069,1139,1153,1077,1086,661,501


In [9]:
pd.concat(SNdict[list(SNdict.keys())[1]])

,0,1,2,3,4,5,6
0,1941,1929,1943,1824,1985,1173,881
0,1141,1141,1162,1202,1187,800,594
0,1015,1140,1152,1211,1183,698,542
0,2223,2264,2368,2238,2339,1369,921


In [10]:
pd.concat(SNdict[list(SNdict.keys())[100]])

,0,1,2,3,4,5,6
0,49,59,48,46,45,27,22
0,658,721,744,674,575,340,259
0,626,614,615,539,530,328,247
0,294,294,277,296,251,124,97
0,635,640,663,660,525,292,191
0,129,144,123,135,115,64,68
0,2,3,2,7,4,1,4
0,37,43,38,36,30,19,22
0,329,319,294,280,250,104,72
0,17,15,9,19,18,8,7


In [ ]:

# D

# D.ENTRIES.hist()